# Generate Asteroid CSV for Calibration Level 2 Data

This pipeline will itterate through the previously generated level 3 asteroid csv to find observation that may contain asteroids. It will determine the level 2 constituents (members) and test if the asteroid present in the level 3 data is viewable in the level 2 sensor data.

## Import Libraries

In [1]:
from astroquery.esa.jwst import Jwst
import numpy as np
from datetime import datetime

from tqdm.notebook import tqdm
tqdm.pandas()

import pandas as pd
from astropy.io import fits
import astropy.time as at
from astroquery.jplhorizons import Horizons
import re
import os
import sys
import logging
import glob
import shapely.wkt
from shapely.geometry import Polygon, Point
from reproject import reproject_interp
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import math as mt
from astropy.time import Time
from astroquery.imcce import Skybot
import astropy.units as u
from astropy.visualization import ZScaleInterval, ImageNormalize,LinearStretch, SqrtStretch, simple_norm
from astropy.nddata import Cutout2D
from PIL import Image
from matplotlib import cm
import numpy as np
from sbident import SBIdent
import time

from scipy import ndimage
from skimage.filters import threshold_otsu
import matplotlib.colors as mcolors

import warnings
warnings.filterwarnings('ignore')

## Utility Functions

In [2]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [3]:
def MJDconversion(modifiedJulianDate):
    #Convert string from Modified Julian Date to YYYMMDD[H:M:S] format
   
    return (Time(modifiedJulianDate, format='mjd').iso)

In [4]:
def isLocatedInImage(targetRA, targetDEC, imagePOLYGON):
    #Check if the RA and DEC coordinate exist within the image bound polygon (returns boolean)
    
    return(imagePOLYGON.contains(Point(targetRA, targetDEC)))

In [5]:
def pullWCS(imagePath):
    #recover data from a fits file
    if not os.path.exists(imagePath):
            return None, None

    with fits.open(imagePath) as hdul: #memmap=True
        header = hdul[1].header
        data = hdul[1].data
        wcs_info = WCS(header)
   
    return(data, wcs_info)

In [6]:
def formatPolygon(polyString):
    #Format the archive polygon string to a format that is compatible with the shapely function
    
    #slice away the polygon charactors 'polygon((' from the start and '))' from the end
    coords = polyString[8:-2].split(' ')
    
    #Add in the fist location at the end to close the loop
    coords.append(coords[0])
    coords.append(coords[1])
    
    return f"POLYGON (({', '.join([coords[i] + ' ' + coords[i+1] for i in range(0, len(coords), 2)])}))"

In [7]:
def proposal2asteroidDict(propList, asteroidList):
    # Create a dictionary where each proposal maps to a set of unique asteroid names
    proposal_dict = {}
    
    # Iterate over the proposals and names simultaneously
    for proposal, asteroids in zip(propList, asteroidList):
        proposal_dict.setdefault(proposal, set()).update(asteroids)
    
    # Convert sets back to lists for the final output
    return {proposal: list(asteroids) for proposal, asteroids in proposal_dict.items()}


In [8]:
def nonZeroCheck(imageData, X, Y):
    #checks to see if any of the asteroid coordinates overlap with the sensor and thus the asteroid has actually measurable data
    # Convert coordinates to integers (floor them)
    X_int = np.floor(X).astype(int)
    Y_int = np.floor(Y).astype(int)

    # Ensure the coordinates are within the bounds of the image
    valid_coords = (X_int >= 0) & (X_int < imageData.shape[1]) & (Y_int >= 0) & (Y_int < imageData.shape[0])

    # Get pixel values at the valid coordinates
    probe_values = imageData[Y_int[valid_coords], X_int[valid_coords]]

    # Check if any of the valid pixel values are non-zero and not NaN
    return np.any((probe_values != 0) & ~np.isnan(probe_values))

In [9]:
def generateFolder(folderName):
    #check if folder exists
    if not os.path.exists(folderName):
        # Folder doesn't exist, so create it
        os.mkdir(folderName)

## Generate ADQL Search

In [10]:
def query4imageMembers(obsList):
    # Prepare the observation IDs for the query
    obsIds = ', '.join([f"'{obsid}'" for obsid in obsList])
    
    # Define the readouts and query filters
    readouts = ['members']
    query_filters = [
        'jwst.archive.calibrationlevel = 3',
        f'jwst.archive.observationid IN ({obsIds})'
    ]
    
    # Construct the query string
    query_string = f"SELECT {', '.join(readouts)} FROM jwst.archive WHERE ({' AND '.join(query_filters)})"
    
    # Launch the job and fetch the results
    job = Jwst.launch_job(query_string, async_job=True)
    result = job.get_results()
    panda_results = result.to_pandas()

    # Extract and process the 'members' column and identify level 2 constituents
    membersListProper = panda_results['members'].str.replace('caom:JWST/', '').str.split(' ').explode().unique()
    
    # Create a dictionary mapping observation IDs to their respective members
    obsMemberDict = panda_results['members'].apply(
        lambda x: x.replace('caom:JWST/', '').split(' ')
    ).to_dict()

    # Return unique members and the observation-member dictionary
    return (membersListProper.tolist(), obsMemberDict)

In [11]:
def level2query(obsList):
    #search the archive for outputs from the level 2 members list    
    obsIds = ', '.join(f"'{obsid}'" for obsid in obsList)
    
    # Define the readouts and query filters
    readouts = [
        'proposal_id', 'observationid', 'dataproducttype', 'intent',
        'target_moving', 'energy_bandpassname',
        'position_bounds_spoly', 'time_bounds_lower', 'time_bounds_upper'
    ]
    query_filters = [
        'jwst.archive.calibrationlevel = 2',
        f"jwst.archive.observationid IN ({obsIds})"
    ]
    
    # Construct the ADQL query string
    query_string = f"SELECT {', '.join(readouts)} FROM jwst.archive WHERE ({' AND '.join(query_filters)})"
    
    # Execute the query and retrieve results
    job = Jwst.launch_job(query_string, async_job=True)
    result = job.get_results()
    df = result.to_pandas()
    
    # Sort the DataFrame by proposal ID and reset the index
    return df.sort_values(by='proposal_id').reset_index(drop=True)

## Pull Lvl3 Asteroid Data

In [12]:
def filterCSV4asteroids(df):    
    # Filter rows where the specified column contains a '[' which contain asteroids
    
    return(df[df['Asteroids'].str.contains(r'\[', na=False)])

In [13]:
def filterCSVbyProposal(propRange, df):
    startingProposal, endingProposal = propRange
    
    #Filter rows based on the proposal range of interest
    filteredDF = df[df['Proposal'].apply(lambda x: startingProposal <= int(x) <= endingProposal)]
    
    return(filteredDF)

In [14]:
def pullCSVdata(CSVloc, propRange):
    # Pull level 3 CSV data
    df = pd.read_csv(CSVloc)
    
    # Filter for rows with asteroids
    dfAsteroids = filterCSV4asteroids(df)
    if dfAsteroids.empty:
        print("Empty Dataframe, No asteroids in CSV")
        return ([], [], [])
    
    # Filter for proposals within the specified range
    dfFiltered = filterCSVbyProposal(propRange, dfAsteroids)
    if dfFiltered.empty:
        print(f"Empty Dataframe, No asteroids in that proposal range {propRange}")
        return ([], [], [])
    
    # Extract relevant columns and process asteroid data in one step
    asteroidListGrouped = dfFiltered['Asteroids'].str.strip('[]').str.replace("'", "").str.split(', ')
    
    # Return the filtered data
    return dfFiltered['Proposal'].to_list(), dfFiltered['Observation'].to_list(), asteroidListGrouped.tolist()

## Output Image Generation

In [15]:
def produceOriginalImage(imageData, WCS, obsID, filePath):
    # Define the output file path
    output_file = f'{filePath}/Original_{obsID}.png'

    # Check if the file already exists to avoid unnecessary plotting
    if not os.path.exists(output_file):
        # Create the figure and axis for plotting
        fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': WCS})

        # Set labels and title
        ax.set_xlabel('RA')
        ax.set_ylabel('DEC')
        ax.set_title(f'{obsID} i2d Image')

        # Normalize image data
        norm = simple_norm(imageData, 'sqrt')

        # Plot the image
        ax.imshow(imageData, cmap='viridis', norm=norm)

        # Save the figure
        plt.savefig(output_file)
        plt.close(fig)

In [16]:
def produceOverlayImage(imageData, WCS, obsID, asteroidList, Xlist, Ylist, filePath):
    overlay_file = f'{filePath}/Overlay_{obsID}.png'
    
    if not os.path.exists(overlay_file):
        # Set figure size based on the number of asteroids
        fig, axes = plt.subplots(1, len(asteroidList) + 1, figsize=(20 + 10 * (len(asteroidList) - 1), 10), 
                                 subplot_kw={'projection': WCS})
        
        # First subplot is for the original image
        ax1 = axes[0]  
        ax1.set_xlabel('RA')
        ax1.set_ylabel('DEC')
        ax1.set_title(f'{obsID} i2d Image')

        # Apply square root normalization for the image
        norm = simple_norm(imageData, 'sqrt')
        ax1.imshow(imageData, cmap='viridis', norm=norm, aspect='auto', zorder=1)

        # Define asteroid-specific colors for plotting
        colors = ['orangered', 'maroon', 'firebrick', 'goldenrod', 'tomato', 'chocolate', 
                  'tab:red', 'tab:orange', 'red', 'orange',  'black', 'yellow']

        # Loop through asteroids and overlay positions, each getting their own subplot zoom
        for indx, asteroid in enumerate(asteroidList):
            ax_ = axes[indx + 1]  # Subsequent subplots for each asteroid
            ax_.set_xlabel('RA')
            ax_.set_ylabel('DEC')
            ax_.set_title(f'{obsID} Asteroid {asteroid}')

            ax_.imshow(imageData, cmap='viridis', norm=norm, aspect='auto', zorder=1)

            X = Xlist[indx]
            Y = Ylist[indx]
            extraPixels = 20  # Buffer space around the asteroid position

            # Get positions of the asteroid bounds
            if len(X) > 1:
                P1 = [X[0], Y[0]]
                P2 = [X[-1], Y[-1]]
            else:
                P1 = [X[0], Y[0]]
                P2 = [X[0], Y[0]]

            # Define the default bounds with extra space 
            lowerx = round(min(P1[0], P2[0])) - extraPixels
            upperx = round(max(P1[0], P2[0])) + extraPixels
            lowery = round(min(P1[1], P2[1])) - extraPixels
            uppery = round(max(P1[1], P2[1])) + extraPixels

            # Plot the bounding box and asteroid positions
            ax1.plot([lowerx, upperx, upperx, lowerx, lowerx], 
                     [lowery, lowery, uppery, uppery, lowery], 
                     alpha=0.6, c=colors[indx], linewidth=2, zorder=2, label=asteroid)
            ax_.scatter(X, Y, s=15, alpha=0.8, c=colors[indx], zorder=2, label=asteroid)
            ax_.set_xlim(lowerx, upperx)
            ax_.set_ylim(lowery, uppery)

        ax1.legend()  # Add legend to the first plot
        plt.tight_layout()  # Automatically adjust subplot parameters for a neat layout

        # Save the overlay image
        plt.savefig(overlay_file)
        plt.close()

In [17]:
def highContrastImage(imageData, WCS, asteroidName, X,Y, path, observation):
    #if not os.path.exists(f'{path}/HighContrastZoom_{observation}_{asteroidName}.png'):
    
    # Define extra padding for the crop
    extra_pixels = 22
    rows, cols = imageData.shape

    # Calculate crop boundaries with bounds checking using numpy.clip
    col_start, col_end = np.clip([int(min(X) - extra_pixels), int(max(X) + extra_pixels)], 0, cols - 1)
    row_start, row_end = np.clip([int(min(Y) - extra_pixels), int(max(Y) + extra_pixels)], 0, rows - 1)

    # Extract cropped region
    cropped_image = imageData[row_start:row_end, col_start:col_end]

    # Plot the high-contrast zoomed image
    plt.figure(figsize=(12, 12))  # Adjusted figure size for better readability
    plt.title(f"{observation} Cutout of Asteroid {asteroidName}")
    plt.xlabel('Pixel Columns')
    plt.ylabel('Pixel Rows')

    # Apply square root normalization for high contrast
    cax = plt.imshow(cropped_image, cmap='viridis')
    plt.colorbar(cax)
    plt.gca().invert_yaxis()

    # Save the figure
    plt.savefig(f'{path}/HighContrastZoom_{observation}_{asteroidName}.png')
    plt.close()
    
    # Return the cropped image
    return cropped_image

In [18]:
def generateSNRimage(imageData, path, observation, asteroidName):
    #Generate a signal to noise plot to demonstrate the cutout of the asteroid area
    # Using   S/N  = (Average Asteroid Signal - Average Background) / (Standard Deviation of Background)
    
    # Mask out zero and NaN values
    masked_image = np.ma.masked_invalid(np.ma.masked_equal(imageData, 0))

    # Extract valid (non-zero and non-NaN) pixel values
    valid_pixels = masked_image.compressed()

    # Apply Otsu's threshold to separate signal from background
    threshold_value = threshold_otsu(valid_pixels)
    signal_pixels = valid_pixels[valid_pixels > threshold_value]
    background_pixels = valid_pixels[valid_pixels <= threshold_value]

    # Calculate mean values of signal and background
    mean_signal = np.mean(signal_pixels)
    mean_background = np.mean(background_pixels)
    std_background = np.std(background_pixels)

    # Compute Signal-to-Noise Ratio (SNR)
    snr = (mean_signal - mean_background) / std_background

    # Create the output visualization
    plt.figure(figsize=(30, 10))

    # Plot 1: Original image with masked areas
    plt.subplot(1, 3, 1)
    plt.title('Original Image with Source')
    cax1 = plt.imshow(masked_image, cmap='gray', zorder=2)
    plt.scatter([0], [0], c='white', label=asteroidName, zorder=1)
    plt.legend()
    plt.colorbar(cax1)
    plt.gca().invert_yaxis()

    # Plot 2: Binary mask using Otsu's threshold
    plt.subplot(1, 3, 2)
    plt.title(f'Mask using Otsu Threshold = {threshold_value:.2f},  S/N = {snr:.2f}')
    cax2 = plt.imshow(masked_image > threshold_value, cmap='gray')
    plt.colorbar(cax2)
    plt.gca().invert_yaxis()

    # Plot 3: 1D histogram of pixel values
    flattened_data = valid_pixels  # Already cleaned and flattened
    hist, bin_edges = np.histogram(flattened_data, bins=50)

    plt.subplot(1, 3, 3)
    plt.bar(bin_edges[:-1], hist, width=np.diff(bin_edges), edgecolor='black', align='edge', zorder = 2)
    plt.title('1D Histogram of Pixel Values')
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')
    plt.grid(True,zorder = 1)

    # Annotate the histogram with key statistics
    plt.axvline(x=mean_background, color='black', linestyle='dashed', label=f'Avg Background = {mean_background:.2f}')
    plt.axvline(x=threshold_value, color='green', linestyle='dashed', label=f'Otsu Threshold = {threshold_value:.2f}')
    plt.axvline(x=mean_signal, color='red', linestyle='dashed', label=f'Avg Signal = {mean_signal:.2f}')
    plt.legend(loc='upper right')

    # Save and close the plot
    output_path = f"{path}/SNR_{observation}_{asteroidName}.png"
    plt.savefig(output_path)
    plt.close()

    return f"{snr:.3f}"

## Asteroid Analysis

In [19]:
def determineAsteroidProperties(asteroidName, expStart, expEnd):
    #pull asteroid properties from JPL
    
    # Query asteroid data from Horizons
    asteroidData = Horizons(
        id=asteroidName, 
        location='Geocentric@JWST', 
        epochs={'start': expStart, 'stop': expEnd, 'step': '1m'}
    )
    
    asteroid_JWST_ephem = asteroidData.ephemerides().to_pandas()
    
    # Extract midpoint index for calculations
    midpoint = len(asteroid_JWST_ephem) // 2
    
    # Extract and format desired properties
    alpha = f"{asteroid_JWST_ephem['alpha'].iloc[midpoint]:.4f}" #Sun-Asteroid-JWST angle
    sun_to_asteroid = f"{asteroid_JWST_ephem['r'].iloc[midpoint]:.8f}"
    asteroid_to_JWST = f"{asteroid_JWST_ephem['delta'].iloc[midpoint]:.8f}"
    
    # Uncertainty (RA and DEC errors)
    position_uncertainty = asteroid_JWST_ephem['RSS_3sigma'].iloc[midpoint]  # combined uncertainty from RA and DEC

    # Category of asteroid based on orbit_class
    asteroid_orbit_class, asteroid_diameter, asteroid_albedo = asteroid_jpl_charactoristics(asteroidName)

    return (alpha, sun_to_asteroid, asteroid_to_JWST, position_uncertainty, asteroid_orbit_class, asteroid_diameter, asteroid_albedo)

In [20]:
def asteroid_jpl_charactoristics(asteroid_name):
    """Collected Manually"""
    #orbital classification classes
    #MB = Main Belt Asteroid
    #IMB = Inner Main Belt Asteroid
    #OMB = Outer Main Belt Asteroid
    #JT = Jupiter Trojan
    #ANEA = Apollo Near Earth Asteroid
    #TNO = TransNeptunian Object
    #C = Centaur
    
    #Diameter estimates from JPL (km)
    #Geometric Albedo estimates from JPL
    
    short_form_dict = {
        'MB':   'Main-Belt Asteroid',
        'JT':   'Jupiter Trojan',
        'IMB':  'Inner Main-Belt Asteroid',
        'ANEA': 'Apollo Near Earth Asteroid',
        'OMB':  'Outer Main-Belt Asteroid',
        'TNO':  'TransNeptunian Object',
        'C':    'Centaur'
    }
        
    
    
    asteroid_params_dict = {'2002 CZ164': ['MB','NA','NA'],
                            '2010 SE55' : ['MB','NA','NA'],
                            '2004 XF41' : ['MB','NA','NA'],
                            '1997 RB1'  : ['MB','3.028','0.305'],
                            '2019 QB44' : ['JT','NA','NA'],
                            '2005 RZ61' : ['MB','NA','NA'],
                            '2008 SX186': ['MB','NA','NA'],
                            '2014 YE88' : ['IMB','NA','NA'],
                            '2008 WE86' : ['MB','NA','NA'],
                            '1999 XY157': ['MB','3.743','0.289'],
                            '2020 KH20' : ['MB','NA','NA'],
                            '2018 HP10' : ['MB','NA','NA'],
                            '2022 WQ19' : ['MB','NA','NA'],
                            '2000 SK247': ['MB','NA','NA'],
                            '2002 UO25' : ['MB','NA','NA'],
                            '2018 AK52' : ['MB','NA','NA'],
                            '2020 OY111': ['MB','NA','NA'],
                            '2012 JS11' : ['ANEA','NA','NA'],
                            '2020 SA57' : ['MB','NA','NA'],
                            '2017 UA86' : ['MB','NA','NA'],
                            '2020 RS115': ['OMB','NA','NA'],
                            '2011 UZ194': ['MB','NA','NA'],
                            '2016 QG127': ['MB','NA','NA'],
                            '1988 RM5'  : ['MB','NA','NA'],
                            '2008 WX'   : ['MB','NA','NA'],
                            '2008 YF34' : ['MB','NA','NA'],
                            '2001 TK17' : ['MB','NA','NA'],
                            '2022 WJ22' : ['MB','NA','NA'],
                            '2021 UE8'  : ['IMB','NA','NA'],
                            '2002 FT2'  : ['MB','6.116','0.068'],
                            '2010 RD101': ['MB','NA','NA'],
                            '2014 DZ80' : ['MB','NA','NA'],
                            '2015 XK95' : ['MB','NA','NA'],
                            '2012 VG116': ['MB','NA','NA'],
                            '2012 SJ16' : ['MB','NA','NA'],
                            '2007 XR66' : ['MB','NA','NA'],
                            '1999 NP39' : ['MB','4.367','0.093'],
                            '1999 XZ93' : ['MB','4.575','0.368'],
                            '1998 BC1'  : ['MB','15.712','0.082'],
                            '2003 EL61' : ['TNO','NA','NA'],
                            '2001 QO160': ['MB','NA','NA'],
                            '2006 DZ86' : ['MB','NA','NA'],
                            '1964 VY'   : ['MB','4.408','0.301'],
                            '2008 EG37' : ['MB','NA','NA'],
                            '1998 QM37' : ['MB','NA','NA'],
                            '2000 WR106': ['TNO','900','0.07'],
                            '2017 PN6': ['MB','NA','NA'],
                            '2016 GT196': ['MB','NA','NA'],
                            '1996 YP': ['MB','NA','NA'],
                            '2001 VF31': ['MB','NA','NA'],
                            '2013 AX137': ['MB','NA','NA'],
                            '2023 RK160': ['MB','NA','NA'],
                            '2003 AB55': ['MB','2.910','0.052'],
                            '2003 QP78': ['MB','2.355','0.046'],
                            '2016 WY66': ['MB','NA','NA'],
                            '2011 BN173': ['MB','NA','NA'],
                            '2023 EL1': ['ANEA','NA','NA'],
                            '2011 RK8': ['MB','NA','NA'],
                            '2008 SV190': ['MB','NA','NA'],
                            '1997 CU26': ['C','NA','NA'],
                            '2002 GG166': ['TNO','NA','NA'],
                            '1999 OX3': ['TNO','NA','NA'],
                            '2000 QC243': ['C','187.5','0.05'],
                            '2015 XN511': ['MB','NA','NA'],
                            '2014 WX577': ['MB','NA','NA'],
                            '2005 SH105': ['MB','NA','NA'],
                            '2007 RG378': ['MB','NA','NA'],
                            '2022 QF262': ['MB','NA','NA'],
                            '1996 NA3': ['MB','NA','NA'],
                            '2001 XZ41': ['MB','NA','NA'],
                            '2006 VT137': ['MB','NA','NA'],
                            '2016 RC17' : ['MB','NA','NA'],
                            '2013 LU28' : ['TNO','NA','NA'],
                            '2000 QC243': ['C','187.5','0.05'],
                            '2000 AY94' : ['MB','NA','NA'],
                            '2015 RG201' : ['MB','NA','NA'],
                            '2008 FA55' : ['MB','NA','NA'],
                            '2003 UM107' : ['MB','NA','NA'],
                            '2011 UK16' : ['MB','NA','NA'],
                            '2011 UM4' : ['MB','NA','NA'],
                            'A907 TC' : ['MB','7.249','0.298'],
                            '2008 TW52' : ['MB','NA','NA'],
                            '2006 XD33' : ['MB','NA','NA'],
                            '2014 OQ381' : ['MB','NA','NA'],
                            '2011 BO29' : ['MB','NA','NA'],
                            '2021 VQ25' : ['MB','NA','NA'],
                            '2014 KY44' : ['IMB','NA','NA'],
                            '2007 TN15' : ['MB','NA','NA'],
                            '2000 SQ146' : ['MB','2.387','0.34'],
                            '2002 GD' : ['MB','NA','NA'],
                            '2016 CJ6' : ['MB','NA','NA'],
                            '2014 NH18' : ['MB','NA','NA'],
                            '2014 WG354' : ['MB','NA','NA']
                            
                           }
    

    phys_params = asteroid_params_dict.get(asteroid_name, None)
    
    if phys_params is None:
        classification, diameter, albedo = None, None, None
    else:
        classification, diameter, albedo = phys_params
        
    return(short_form_dict.get(classification, None), diameter, albedo)

## Asteroid Detection

In [21]:
def jplHorizonsSearch(targetID, startTime, stopTime, polyString):
    #Search the JPL Horizons data for a specific target to get orbital values. 
    #This method is more accurate then the cone search and provides a double check for asteroids (named from the cone search) existing in the image
    
    #generate polygon variable from the polygon string
    poly = shapely.wkt.loads(polyString)
    
    #define the probe minutes at its lowest setting as to not miss ny additional data
    #Note: there is likely a way to toggle this depending on the length of the exposure time as sometimes the exposure tis very long
    probeMinutes = 1
    
    #get return from the JPL horizons output regarding the specific target searched
    jpl_output = Horizons(
        id=targetID, 
        location='Geocentric@JWST', 
        epochs={'start': str(startTime), 'stop' : str(stopTime), 'step' : f"{probeMinutes}m"}
    ) 

    #get Ephemerides data
    jpl_pandas = jpl_output.ephemerides().to_pandas()
    
    #pull out the RA and DEC lists marking the position of the asteroid at this time
    asteroid_positions = zip(jpl_pandas['RA'], jpl_pandas['DEC'])
    
    #check to see if there are any times during this that the asteroid intercepts with the imaging window
    return (jpl_pandas['RA'], jpl_pandas['DEC'], any(isLocatedInImage(ra, dec, poly) for ra, dec in asteroid_positions))

In [22]:
def asteroidsearch(row, folderName, level3obs, asteroidList):
    proposal = row['proposal_id']
    observation = row['observationid']
    #datatype = row['dataproducttype']
    intent = row['intent']
    #moving = row['target_moving']
    polygonString = row['position_bounds_spoly']
    expStartMJD = row['time_bounds_lower']
    expEndMJD = row['time_bounds_upper']
    
    #Generate output folder based on proposal
    propPath = f'{folderName}/{proposal}'
    
    #Generate level 3 and level 2 folders
    lvl3path = f'{propPath}/Level3_Products'
    
    lvl2path = f'{propPath}/Level2_Members'
    
    #reformat the image polygon
    polyStringFmt =  str(formatPolygon(str(polygonString)))
    
    #convertTime format
    expStart = (MJDconversion(expStartMJD))
    expEnd   = (MJDconversion(expEndMJD))
    
    
    #there might be a way to use if not os.path.exists(f'{filePath}/Original_{obsID}.png'):
    #to see if the observation has already been tested and save on computation time
    containedAsteroidNames = []
    asteroidsNoData = []
    containedAsteroidX = []
    containedAsteroidY = []
    usefulAsteroids = False
    
    for asteroid in asteroidList:
        raList, decList, containedCheck = jplHorizonsSearch(asteroid, expStart, expEnd, polyStringFmt)
        
        if containedCheck:
            #pull the WCS and image data for the observation
            jwst_file = f"jw0{next(c for c in proposal if c != '0')}"
            dataPath = f'/data/user/jwst_{jwst_file}/jw0{proposal}/{observation}_i2d.fits.gz'
            imageData, WCS = pullWCS(dataPath)
            
            #check that the data exists
            if WCS is None:
                print(f"ERROR: Image Data from {dataPath} Not Found")
                asteroidsNoData.append(asteroid)
                
            else:
                #convert RA and DEC to pixel locations
                asteroidPixelX, asteroidPixelY = WCS.all_world2pix(raList, decList, 0)
                
                #perform the non zero check to ensure that the asteroid travels across the sensor and not the dead area
                nonZero = nonZeroCheck(imageData, asteroidPixelX, asteroidPixelY)
                
                if nonZero:
                    containedAsteroidNames.append(asteroid)
                    containedAsteroidX.append(asteroidPixelX)
                    containedAsteroidY.append(asteroidPixelY)
                    usefulAsteroids = True
    
    
    if usefulAsteroids:
        generateFolder(propPath)
        generateFolder(lvl2path)
    
        produceOverlayImage(imageData, WCS, observation, containedAsteroidNames, containedAsteroidX, containedAsteroidY, lvl2path)
        produceOriginalImage(imageData, WCS, observation, lvl2path)

        snrList = []
        phaseList = []
        sun_asteroid_dist = []
        asteroid_JWST_dist = []
        pos_err_list = []
        classification_list = []
        diameter_list = []
        albedo_list = []
        
        for indx in range(len(containedAsteroidNames)):
            croppedImg = highContrastImage(imageData, WCS, containedAsteroidNames[indx], containedAsteroidX[indx], containedAsteroidY[indx], lvl2path, observation)
            
            if not isinstance(croppedImg, bool):
                snr = generateSNRimage(croppedImg, lvl2path, observation, containedAsteroidNames[indx])
                snrList.append(snr)
                
                phaseAngle, sun2asteroid, asteroid2JWST, pos_err, orbit_classification, diameter, albedo = determineAsteroidProperties(containedAsteroidNames[indx], expStart, expEnd)
                
                phaseList.append(phaseAngle)
                sun_asteroid_dist.append(sun2asteroid)
                asteroid_JWST_dist.append(asteroid2JWST)
                pos_err_list.append(pos_err)
                classification_list.append(orbit_classification)
                diameter_list.append(diameter)
                albedo_list.append(albedo)
                

        if len(asteroidsNoData) > 0:  
            containedAsteroidNames.extend([a + "*" for a in asteroidsNoData])
            snrList.extend(['No Data' for a in range(len(asteroidsNoData))])
            phaseList.extend(['No Data' for a in range(len(asteroidsNoData))])
            sun_asteroid_dist.extend(['No Data' for a in range(len(asteroidsNoData))])
            asteroid_JWST_dist.extend(['No Data' for a in range(len(asteroidsNoData))])
            pos_err_list.extend(['No Data' for a in range(len(asteroidsNoData))])
            classification_list.extend(['No Data' for a in range(len(asteroidsNoData))])          
            diameter_list.extend(['No Data' for a in range(len(asteroidsNoData))])  
            albedo_list.extend(['No Data' for a in range(len(asteroidsNoData))])  
                
    else:
        if len(asteroidsNoData) > 0:
            containedAsteroidNames = [a + "*" for a in asteroidsNoData]
            snrList = 'No Data'
            phaseList = 'No Data'
            sun_asteroid_dist = 'No Data'
            asteroid_JWST_dist = 'No Data'
            pos_err_list = 'No Data'
            classification_list = 'No Data'
            diameter_list = 'No Data' 
            albedo_list = 'No Data'
            
        else:
            containedAsteroidNames = ''
            snrList = ''
            phaseList = ''
            sun_asteroid_dist =''
            asteroid_JWST_dist =''
            pos_err_list = ''
            classification_list = ''
            diameter_list = '' 
            albedo_list = ''
            
    return pd.Series([intent, expStart, expEnd, containedAsteroidNames, classification_list, snrList, phaseList, sun_asteroid_dist, asteroid_JWST_dist, diameter_list, albedo_list, pos_err_list], index=['Image Intent', 'Exposure Start', 'Exposure End', 'Asteroids', 'JPL Classification', 'S/N', 'JPL Phase Angle (Deg)', 'JPL Sun-Asteroid Dist (AU)', 'JPL Asteroid-JWST Dist (AU)', 'JPL Diameter (km)', 'JPL Geo Albedo', 'JPL Pos Uncertainty (Arcsec)'])

## Main Function

In [23]:
def main(csvName, folderName, propRange, username, password):
    #main function for the lvl 2 asteroid detection and saving to the csv
    
    with HiddenPrints():
        Jwst.login(user=f"{username}", password=f"{password}")
        
        generateFolder(folderName)
    
    #pull the useful asteroid rows from the provided lvl 3 csv
    proposals, lvl3observations, asteroids = pullCSVdata(csvName, propRange)
    
    asteroidDict = proposal2asteroidDict(proposals, asteroids)

    if proposals and lvl3observations:
        uniqueProposals = sorted(list(set(proposals)))
        
        print(f"From proposals {uniqueProposals} found {len(lvl3observations)} Observations containing asteroids")
        
        #search for the members of the level 3 images 
        with HiddenPrints():
            members, obsMemberDict = query4imageMembers(lvl3observations)
        
        #find the archive info on all of the unique members
        with HiddenPrints():
            level2df = level2query(members)
        
        
        #itterate through the dataframe using the row as the input to a search function and add 2 columns to the df
        asteroidCSV = (
            level2df
            .assign(
                **level2df.progress_apply(
                    lambda row: asteroidsearch(row, folderName, lvl3observations, asteroidDict.get(int(row['proposal_id']))), axis = 1
                )
            )
        )

        asteroidCSV = (
            asteroidCSV[
                ['proposal_id', 'observationid', 'energy_bandpassname', 'target_moving',
                 'Image Intent', 'position_bounds_spoly', 'Exposure Start', 'Exposure End', 
                 'Asteroids', 'JPL Classification', 'S/N', 'JPL Phase Angle (Deg)', 'JPL Sun-Asteroid Dist (AU)', 
                 'JPL Asteroid-JWST Dist (AU)', 'JPL Diameter (km)', 'JPL Geo Albedo',
                 'JPL Pos Uncertainty (Arcsec)']
            ]
            .rename(columns={
                'proposal_id': 'Proposal', 
                'observationid': 'Observation', 
                'energy_bandpassname': 'Filter', 
                'target_moving': 'Moving', 
                'position_bounds_spoly': 'Polygon', 
                'Exposure Start': 'Exp Start', 
                'Exposure End': 'Exp End', 
            })
            .assign(Moving=lambda df: df['Moving'].replace({0: 'No', 1: 'Yes'}))
            .sort_values(by='Observation', ascending=True)
            .reset_index(drop=True)
        )
        
        #pd.set_option('display.max_rows', None) 
        #pd.set_option('display.max_columns', None)
        #pd.set_option('display.width', None)

        #print(f'{folderName}/LVL2_Asteroids_{propRange[0]}_{propRange[1]}.csv')
        #csvName = f'{folderName}/LVL2_Asteroids_{propRange[0]}_{propRange[1]}.csv'
        #asteroidCSV.to_csv(csvName, index=False)
        
        return(asteroidCSV)

## Run

In [24]:
%%time

# Constants and parameters
username = "nmartind"
password = "Mr.Fantastic1999"
starting_proposal = 1000
ending_proposal = 2000
output_folder = "MIRI"
input_csv = "LVL3_Asteroids/Prop1000-2000/LVL3_Full.csv"
propStepSize = 5

# List to hold CSV segments
csv_lists = []

current = starting_proposal

while current <= ending_proposal:
    segment_end = min(current+propStepSize-1,ending_proposal)
    
    csv_segment = main(input_csv, output_folder, [int(current), int(segment_end)], username, password)

    
    if csv_segment is not None and not csv_segment.empty:
        csvName = f'{output_folder}/LVL2_Asteroids_Seg_{current}.csv'
        csv_segment.to_csv(csvName, index=False)
        csv_lists.append(csv_segment)
        print('saved')
        
    current += propStepSize   
    
if len(csv_lists) > 0:
    combined_csv = pd.concat(csv_lists, ignore_index=True)

    csvName = f'{output_folder}/LVL2_Asteroids_{starting_proposal}_{ending_proposal}.csv'
    combined_csv.to_csv(csvName, index=False)

    print(f'Saving to {csvName}')
    
else:
    print('No data in that range')

Empty Dataframe, No asteroids in that proposal range [1000, 1004]
Empty Dataframe, No asteroids in that proposal range [1005, 1009]
Empty Dataframe, No asteroids in that proposal range [1010, 1014]
Empty Dataframe, No asteroids in that proposal range [1015, 1019]
From proposals [1022] found 2 Observations containing asteroids


  0%|          | 0/2 [00:00<?, ?it/s]

saved
Empty Dataframe, No asteroids in that proposal range [1025, 1029]
Empty Dataframe, No asteroids in that proposal range [1030, 1034]
Empty Dataframe, No asteroids in that proposal range [1035, 1039]
Empty Dataframe, No asteroids in that proposal range [1040, 1044]
Empty Dataframe, No asteroids in that proposal range [1045, 1049]
From proposals [1051] found 34 Observations containing asteroids


  0%|          | 0/102 [00:00<?, ?it/s]

saved
Empty Dataframe, No asteroids in that proposal range [1055, 1059]
Empty Dataframe, No asteroids in that proposal range [1060, 1064]
Empty Dataframe, No asteroids in that proposal range [1065, 1069]
Empty Dataframe, No asteroids in that proposal range [1070, 1074]
Empty Dataframe, No asteroids in that proposal range [1075, 1079]
Empty Dataframe, No asteroids in that proposal range [1080, 1084]
Empty Dataframe, No asteroids in that proposal range [1085, 1089]
Empty Dataframe, No asteroids in that proposal range [1090, 1094]
Empty Dataframe, No asteroids in that proposal range [1095, 1099]
Empty Dataframe, No asteroids in that proposal range [1100, 1104]
Empty Dataframe, No asteroids in that proposal range [1105, 1109]
Empty Dataframe, No asteroids in that proposal range [1110, 1114]
Empty Dataframe, No asteroids in that proposal range [1115, 1119]
Empty Dataframe, No asteroids in that proposal range [1120, 1124]
Empty Dataframe, No asteroids in that proposal range [1125, 1129]
Empt

...

saved
Empty Dataframe, No asteroids in that proposal range [1805, 1809]
Empty Dataframe, No asteroids in that proposal range [1810, 1814]
Empty Dataframe, No asteroids in that proposal range [1815, 1819]
Empty Dataframe, No asteroids in that proposal range [1820, 1824]
Empty Dataframe, No asteroids in that proposal range [1825, 1829]
Empty Dataframe, No asteroids in that proposal range [1830, 1834]
From proposals [1837] found 20 Observations containing asteroids


  0%|          | 0/735 [00:00<?, ?it/s]

saved
Empty Dataframe, No asteroids in that proposal range [1840, 1844]
Empty Dataframe, No asteroids in that proposal range [1845, 1849]
From proposals [1854] found 2 Observations containing asteroids


  0%|          | 0/16 [00:00<?, ?it/s]

saved
Empty Dataframe, No asteroids in that proposal range [1855, 1859]
Empty Dataframe, No asteroids in that proposal range [1860, 1864]
Empty Dataframe, No asteroids in that proposal range [1865, 1869]
Empty Dataframe, No asteroids in that proposal range [1870, 1874]
Empty Dataframe, No asteroids in that proposal range [1875, 1879]
Empty Dataframe, No asteroids in that proposal range [1880, 1884]
Empty Dataframe, No asteroids in that proposal range [1885, 1889]
Empty Dataframe, No asteroids in that proposal range [1890, 1894]
Empty Dataframe, No asteroids in that proposal range [1895, 1899]
Empty Dataframe, No asteroids in that proposal range [1900, 1904]
Empty Dataframe, No asteroids in that proposal range [1905, 1909]
Empty Dataframe, No asteroids in that proposal range [1910, 1914]
Empty Dataframe, No asteroids in that proposal range [1915, 1919]
Empty Dataframe, No asteroids in that proposal range [1920, 1924]
Empty Dataframe, No asteroids in that proposal range [1925, 1929]
Empt

In [25]:
%%time

# Constants and parameters
username = "nmartind"
password = "Mr.Fantastic1999"
starting_proposal = 2000
ending_proposal = 3000
output_folder = "MIRI"
input_csv = "LVL3_Asteroids/Prop2000-3000/LVL3_Full.csv"
propStepSize = 5

# List to hold CSV segments
csv_lists = []

current = starting_proposal

while current <= ending_proposal:
    segment_end = min(current+propStepSize-1,ending_proposal)
    
    csv_segment = main(input_csv, output_folder, [int(current), int(segment_end)], username, password)

    
    if csv_segment is not None and not csv_segment.empty:
        csvName = f'{output_folder}/LVL2_Asteroids_Seg_{current}.csv'
        csv_segment.to_csv(csvName, index=False)
        csv_lists.append(csv_segment)
        print('saved')
        
    current += propStepSize   
    
if len(csv_lists) > 0:
    combined_csv = pd.concat(csv_lists, ignore_index=True)

    csvName = f'{output_folder}/LVL2_Asteroids_{starting_proposal}_{ending_proposal}.csv'
    combined_csv.to_csv(csvName, index=False)

    print(f'Saving to {csvName}')
    
else:
    print('No data in that range')

Empty Dataframe, No asteroids in that proposal range [2000, 2004]
Empty Dataframe, No asteroids in that proposal range [2005, 2009]
Empty Dataframe, No asteroids in that proposal range [2010, 2014]
From proposals [2016] found 3 Observations containing asteroids


  0%|          | 0/12 [00:00<?, ?it/s]

saved
Empty Dataframe, No asteroids in that proposal range [2020, 2024]
From proposals [2025] found 1 Observations containing asteroids


  0%|          | 0/12 [00:00<?, ?it/s]

saved
Empty Dataframe, No asteroids in that proposal range [2030, 2034]
Empty Dataframe, No asteroids in that proposal range [2035, 2039]
Empty Dataframe, No asteroids in that proposal range [2040, 2044]
From proposals [2046] found 8 Observations containing asteroids


...

ERROR: Image Data from /data/user/jwst_jw02/jw02820/jw02820013001_02101_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw02/jw02820/jw02820013001_02101_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw02/jw02820/jw02820029001_03102_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw02/jw02820/jw02820023001_02101_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw02/jw02820/jw02820029001_03103_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw02/jw02820/jw02820029001_03101_00001_mirimage_i2d.fits.gz Not Found
saved
Empty Dataframe, No asteroids in that proposal range [2825, 2829]
Empty Dataframe, No asteroids in that proposal range [2830, 2834]
Empty Dataframe, No asteroids in that proposal range [2835, 2839]
Empty Dataframe, No asteroids in that proposal range [2840, 2844]
Empty Dataframe, No asteroids in that proposal range [2845, 2849]
Empty Data

  0%|          | 0/26 [00:00<?, ?it/s]

ERROR: Image Data from /data/user/jwst_jw02/jw02970/jw02970013001_03106_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw02/jw02970/jw02970013001_03102_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw02/jw02970/jw02970013001_03104_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw02/jw02970/jw02970013001_03102_00002_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw02/jw02970/jw02970011001_05101_00002_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw02/jw02970/jw02970011001_09101_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw02/jw02970/jw02970011001_07101_00004_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw02/jw02970/jw02970011001_07101_00003_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw02/jw02970/jw02970013001_03104_00004_mirimage_i2d.fits.gz Not Found
ERROR: Image Data f

  0%|          | 0/256 [00:00<?, ?it/s]

saved
Empty Dataframe, No asteroids in that proposal range [2990, 2994]
Empty Dataframe, No asteroids in that proposal range [2995, 2999]
Empty Dataframe, No asteroids in that proposal range [3000, 3000]
Saving to MIRI/LVL2_Asteroids_2000_3000.csv
CPU times: user 6min 41s, sys: 1min 29s, total: 8min 11s
Wall time: 12min 36s


In [26]:
%%time

# Constants and parameters
username = "nmartind"
password = "Mr.Fantastic1999"
starting_proposal = 3000
ending_proposal = 4000
output_folder = "MIRI"
input_csv = "LVL3_Asteroids/Prop3000-4000/LVL3_Full.csv"
propStepSize = 5

# List to hold CSV segments
csv_lists = []

current = starting_proposal

while current <= ending_proposal:
    segment_end = min(current+propStepSize-1,ending_proposal)
    
    csv_segment = main(input_csv, output_folder, [int(current), int(segment_end)], username, password)

    
    if csv_segment is not None and not csv_segment.empty:
        csvName = f'{output_folder}/LVL2_Asteroids_Seg_{current}.csv'
        csv_segment.to_csv(csvName, index=False)
        csv_lists.append(csv_segment)
        print('saved')
        
    current += propStepSize   
    
if len(csv_lists) > 0:
    combined_csv = pd.concat(csv_lists, ignore_index=True)

    csvName = f'{output_folder}/LVL2_Asteroids_{starting_proposal}_{ending_proposal}.csv'
    combined_csv.to_csv(csvName, index=False)

    print(f'Saving to {csvName}')
    
else:
    print('No data in that range')

Empty Dataframe, No asteroids in that proposal range [3000, 3004]
Empty Dataframe, No asteroids in that proposal range [3005, 3009]
Empty Dataframe, No asteroids in that proposal range [3010, 3014]
Empty Dataframe, No asteroids in that proposal range [3015, 3019]
Empty Dataframe, No asteroids in that proposal range [3020, 3024]
Empty Dataframe, No asteroids in that proposal range [3025, 3029]
From proposals [3034] found 4 Observations containing asteroids


  0%|          | 0/52 [00:00<?, ?it/s]

ERROR: Image Data from /data/user/jwst_jw03/jw03034/jw03034023001_03102_00003_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03034/jw03034020001_03104_00003_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03034/jw03034002001_03106_00003_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03034/jw03034020001_03104_00002_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03034/jw03034020001_03102_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03034/jw03034020001_03104_00004_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03034/jw03034020001_03102_00003_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03034/jw03034002001_03104_00003_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03034/jw03034020001_03102_00004_mirimage_i2d.fits.gz Not Found
ERROR: Image Data f

  0%|          | 0/52 [00:00<?, ?it/s]

ERROR: Image Data from /data/user/jwst_jw03/jw03228/jw03228113001_08101_00002_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03228/jw03228003001_03104_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03228/jw03228003001_03102_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03228/jw03228113001_08101_00004_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03228/jw03228003001_03102_00003_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03228/jw03228004001_03104_00002_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03228/jw03228121001_06101_00003_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03228/jw03228121001_06101_00004_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03228/jw03228004001_03106_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data f

...

saved
Empty Dataframe, No asteroids in that proposal range [3710, 3714]
Empty Dataframe, No asteroids in that proposal range [3715, 3719]
Empty Dataframe, No asteroids in that proposal range [3720, 3724]
Empty Dataframe, No asteroids in that proposal range [3725, 3729]
Empty Dataframe, No asteroids in that proposal range [3730, 3734]
Empty Dataframe, No asteroids in that proposal range [3735, 3739]
Empty Dataframe, No asteroids in that proposal range [3740, 3744]
Empty Dataframe, No asteroids in that proposal range [3745, 3749]
Empty Dataframe, No asteroids in that proposal range [3750, 3754]
Empty Dataframe, No asteroids in that proposal range [3755, 3759]
Empty Dataframe, No asteroids in that proposal range [3760, 3764]
Empty Dataframe, No asteroids in that proposal range [3765, 3769]
Empty Dataframe, No asteroids in that proposal range [3770, 3774]
Empty Dataframe, No asteroids in that proposal range [3775, 3779]
Empty Dataframe, No asteroids in that proposal range [3780, 3784]
Empt

  0%|          | 0/26 [00:00<?, ?it/s]

ERROR: Image Data from /data/user/jwst_jw03/jw03886/jw03886001001_09101_00003_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03886/jw03886001001_05101_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03886/jw03886001001_07101_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03886/jw03886001001_05101_00004_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03886/jw03886001001_07101_00002_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03886/jw03886018001_03106_00002_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03886/jw03886018001_03104_00003_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03886/jw03886018001_03102_00003_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03886/jw03886018001_03106_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data f

  0%|          | 0/4 [00:00<?, ?it/s]

ERROR: Image Data from /data/user/jwst_jw03/jw03954/jw03954017001_02101_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03954/jw03954017001_02101_00002_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03954/jw03954017001_02101_00004_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw03/jw03954/jw03954017001_02101_00003_mirimage_i2d.fits.gz Not Found
saved
Empty Dataframe, No asteroids in that proposal range [3955, 3959]
Empty Dataframe, No asteroids in that proposal range [3960, 3964]
Empty Dataframe, No asteroids in that proposal range [3965, 3969]
Empty Dataframe, No asteroids in that proposal range [3970, 3974]
Empty Dataframe, No asteroids in that proposal range [3975, 3979]
Empty Dataframe, No asteroids in that proposal range [3980, 3984]
Empty Dataframe, No asteroids in that proposal range [3985, 3989]
Empty Dataframe, No asteroids in that proposal range [3990, 3994]
Empty Dataframe, No asteroids 

In [27]:
%%time

# Constants and parameters
username = "nmartind"
password = "Mr.Fantastic1999"
starting_proposal = 4000
ending_proposal = 5000
output_folder = "MIRI"
input_csv = "LVL3_Asteroids/Prop4000-5000/LVL3_Full.csv"
propStepSize = 5

# List to hold CSV segments
csv_lists = []

current = starting_proposal

while current <= ending_proposal:
    segment_end = min(current+propStepSize-1,ending_proposal)
    
    csv_segment = main(input_csv, output_folder, [int(current), int(segment_end)], username, password)

    
    if csv_segment is not None and not csv_segment.empty:
        csvName = f'{output_folder}/LVL2_Asteroids_Seg_{current}.csv'
        csv_segment.to_csv(csvName, index=False)
        csv_lists.append(csv_segment)
        print('saved')
        
    current += propStepSize   
    
if len(csv_lists) > 0:
    combined_csv = pd.concat(csv_lists, ignore_index=True)

    csvName = f'{output_folder}/LVL2_Asteroids_{starting_proposal}_{ending_proposal}.csv'
    combined_csv.to_csv(csvName, index=False)

    print(f'Saving to {csvName}')
    
else:
    print('No data in that range')

Empty Dataframe, No asteroids in that proposal range [4000, 4004]
Empty Dataframe, No asteroids in that proposal range [4005, 4009]
Empty Dataframe, No asteroids in that proposal range [4010, 4014]
Empty Dataframe, No asteroids in that proposal range [4015, 4019]
Empty Dataframe, No asteroids in that proposal range [4020, 4024]
Empty Dataframe, No asteroids in that proposal range [4025, 4029]
Empty Dataframe, No asteroids in that proposal range [4030, 4034]
Empty Dataframe, No asteroids in that proposal range [4035, 4039]
Empty Dataframe, No asteroids in that proposal range [4040, 4044]
Empty Dataframe, No asteroids in that proposal range [4045, 4049]
Empty Dataframe, No asteroids in that proposal range [4050, 4054]
Empty Dataframe, No asteroids in that proposal range [4055, 4059]
Empty Dataframe, No asteroids in that proposal range [4060, 4064]
Empty Dataframe, No asteroids in that proposal range [4065, 4069]
Empty Dataframe, No asteroids in that proposal range [4070, 4074]
Empty Data

  0%|          | 0/16 [00:00<?, ?it/s]

saved
Empty Dataframe, No asteroids in that proposal range [4260, 4264]
Empty Dataframe, No asteroids in that proposal range [4265, 4269]
Empty Dataframe, No asteroids in that proposal range [4270, 4274]
From proposals [4278] found 3 Observations containing asteroids


  0%|          | 0/6 [00:00<?, ?it/s]

ERROR: Image Data from /data/user/jwst_jw04/jw04278/jw04278014001_04101_00002_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw04/jw04278/jw04278014001_06101_00002_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw04/jw04278/jw04278014001_02101_00002_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw04/jw04278/jw04278014001_06101_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw04/jw04278/jw04278014001_02101_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw04/jw04278/jw04278014001_04101_00001_mirimage_i2d.fits.gz Not Found
saved
Empty Dataframe, No asteroids in that proposal range [4280, 4284]
Empty Dataframe, No asteroids in that proposal range [4285, 4289]
From proposals [4290] found 1 Observations containing asteroids


...

saved
Empty Dataframe, No asteroids in that proposal range [4295, 4299]
Empty Dataframe, No asteroids in that proposal range [4300, 4304]
Empty Dataframe, No asteroids in that proposal range [4305, 4309]
Empty Dataframe, No asteroids in that proposal range [4310, 4314]
Empty Dataframe, No asteroids in that proposal range [4315, 4319]
Empty Dataframe, No asteroids in that proposal range [4320, 4324]
Empty Dataframe, No asteroids in that proposal range [4325, 4329]
Empty Dataframe, No asteroids in that proposal range [4330, 4334]
Empty Dataframe, No asteroids in that proposal range [4335, 4339]
Empty Dataframe, No asteroids in that proposal range [4340, 4344]
Empty Dataframe, No asteroids in that proposal range [4345, 4349]
Empty Dataframe, No asteroids in that proposal range [4350, 4354]
Empty Dataframe, No asteroids in that proposal range [4355, 4359]
Empty Dataframe, No asteroids in that proposal range [4360, 4364]
Empty Dataframe, No asteroids in that proposal range [4365, 4369]
Empt

  0%|          | 0/13 [00:00<?, ?it/s]

ERROR: Image Data from /data/user/jwst_jw04/jw04516/jw04516009001_02101_00001_mirimage_i2d.fits.gz Not Found
saved
Empty Dataframe, No asteroids in that proposal range [4520, 4524]
Empty Dataframe, No asteroids in that proposal range [4525, 4529]
Empty Dataframe, No asteroids in that proposal range [4530, 4534]
Empty Dataframe, No asteroids in that proposal range [4535, 4539]
Empty Dataframe, No asteroids in that proposal range [4540, 4544]
Empty Dataframe, No asteroids in that proposal range [4545, 4549]
Empty Dataframe, No asteroids in that proposal range [4550, 4554]
Empty Dataframe, No asteroids in that proposal range [4555, 4559]
Empty Dataframe, No asteroids in that proposal range [4560, 4564]
Empty Dataframe, No asteroids in that proposal range [4565, 4569]
Empty Dataframe, No asteroids in that proposal range [4570, 4574]
Empty Dataframe, No asteroids in that proposal range [4575, 4579]
Empty Dataframe, No asteroids in that proposal range [4580, 4584]
Empty Dataframe, No asteroi

  0%|          | 0/13 [00:00<?, ?it/s]

ERROR: Image Data from /data/user/jwst_jw04/jw04727/jw04727008001_04104_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw04/jw04727/jw04727008001_02101_00001_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw04/jw04727/jw04727008001_04102_00003_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw04/jw04727/jw04727008001_04102_00002_mirimage_i2d.fits.gz Not Found
ERROR: Image Data from /data/user/jwst_jw04/jw04727/jw04727008001_04102_00001_mirimage_i2d.fits.gz Not Found
saved
Empty Dataframe, No asteroids in that proposal range [4730, 4734]
Empty Dataframe, No asteroids in that proposal range [4735, 4739]
Empty Dataframe, No asteroids in that proposal range [4740, 4744]
Empty Dataframe, No asteroids in that proposal range [4745, 4749]
Empty Dataframe, No asteroids in that proposal range [4750, 4754]
Empty Dataframe, No asteroids in that proposal range [4755, 4759]
Empty Dataframe, No asteroids in that proposal range 